#### Define your own model

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

# use nn.Sequential to realize a MLP
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.1272,  0.0367,  0.1830, -0.0094,  0.0232, -0.0538,  0.1229, -0.4048,
         -0.1237, -0.1916],
        [-0.1796,  0.1064,  0.3098, -0.0925,  0.0335, -0.0627,  0.1714, -0.3958,
         -0.0747, -0.1882]], grad_fn=<AddmmBackward0>)

In [2]:
# define our own MLP
class MLP(nn.Module):
    def __init__(self):
        # init the super class
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    # define the forward function
    def forward(self, X):
        # use relu as the activation function
        return self.out(F.relu(self.hidden(X)))

In [3]:
net = MLP()
net(X)

tensor([[-0.1344, -0.2172,  0.0795, -0.1472,  0.0561, -0.2207, -0.0029, -0.0156,
         -0.0361,  0.2012],
        [-0.1714, -0.2731,  0.0183, -0.1128,  0.1023, -0.2176,  0.0658,  0.0205,
         -0.0513,  0.0793]], grad_fn=<AddmmBackward0>)

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # _modules is an OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict guarantees the order of the modules
        for block in self._modules.values():
            X = block(X)
        return X

In [5]:
# use MySequential to realize a MLP (same as sequential)
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0183, -0.0784, -0.0632,  0.1604,  0.0069,  0.1568, -0.0386, -0.0742,
          0.1490,  0.1412],
        [-0.0203, -0.0935,  0.0202,  0.1756,  0.0812,  0.0619, -0.0007, -0.0979,
          0.1251,  0.0975]], grad_fn=<AddmmBackward0>)

In [6]:
# when we have a constant parameter
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # define a constant parameter
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # share two linear layer
        X = self.linear(X)
        # some random code of a control flow
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [7]:
net = FixedHiddenMLP()
net(X)

tensor(0.1202, grad_fn=<SumBackward0>)

In [8]:
# we can also do some nesting operation
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0706, grad_fn=<SumBackward0>)

#### Check your parameters

In [9]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.0201],
        [0.0677]], grad_fn=<AddmmBackward0>)

In [10]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.1772,  0.1254,  0.0613, -0.0507,  0.2648,  0.2830,  0.0583,  0.2798]])), ('bias', tensor([-0.1076]))])


In [11]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.1076], requires_grad=True)
tensor([-0.1076])


In [12]:
# we did not call the grad function, so the grad is None
net[2].weight.grad == None

True

In [13]:
# 这里用了序列解包的写法
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [14]:
# or we can access with the following way
net.state_dict()['2.bias'].data

tensor([-0.1076])

In [15]:
# use loop to define network
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # loop here
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.0091],
        [0.0091]], grad_fn=<AddmmBackward0>)

In [16]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [17]:
# use multilayer index to access the parameter
rgnet[0][1][0].bias.data

tensor([-0.0561, -0.1405, -0.2975,  0.0098, -0.3325, -0.1047, -0.1833, -0.1100])

##### Initialize the parameters

In [18]:
# init with normal distribution
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0117, -0.0009, -0.0054,  0.0119]), tensor(0.))

In [19]:
# init with constant
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [20]:
# we can apply different init function to different layer
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([0.4280, 0.6211, 0.3868, 0.0114])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [21]:
# we can also define our own init function
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000, -7.2780,  0.0000,  9.1759],
        [-9.6885,  7.6889,  5.1641,  0.0000]], grad_fn=<SliceBackward0>)

In [22]:
# share a name to share the parameters
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# check the parameters of the shared layer
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# when we change the parameter of one layer, the other layer will also change
# at the same time, gradient will add up when BP
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


#### Self-define Layer

In [23]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

In [24]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [25]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [26]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(4.6566e-09, grad_fn=<MeanBackward0>)

In [27]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [28]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-1.4166,  2.6053, -0.0677],
        [-1.1027, -0.6444,  1.1846],
        [-0.2875,  0.1051, -0.6924],
        [-2.4912,  1.2359,  1.0760],
        [ 0.7151, -1.0676, -1.2086]], requires_grad=True)

In [29]:
linear(torch.rand(2, 5))

tensor([[0.0000, 0.1589, 0.0000],
        [0.0000, 1.3720, 0.0000]])

In [30]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[3.4906],
        [0.0000]])